# I. Install Package

In [ ]:
!pip install cmake
!pip install dlib
!pip install face_recognition
!pip install face_alignment
!pip install face_recognition_models
!pip install mtcnn
!pip install pymysql

## II. Google Colab Mount

In [ ]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')

# III. Import Package

In [ ]:
import cv2
import cmake
import dlib
import face_recognition_models
#face_recognition 전에 GPU로 변경해야 함
import face_recognition
import face_alignment
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import pyplot
from mtcnn.mtcnn import MTCNN
import matplotlib.image as img
import pymysql
import itertools

# IV. 동일 인물 검출

## i) Face Crop From Source Function

In [ ]:
def get_cropimg(filename):
  detector = MTCNN()
  result_list= detector.detect_faces(pyplot.imread(filename))
  res = []
  imgNum = 0
  data = plt.imread(filename)
  for i in range(len(result_list)):
    result_list2=[]    
    for v in result_list[i]['box']:
      if v >=0:
        result_list2.append(v)
      else:
        result_list2.append(0)
    x1, y1, width, height = result_list2
    x2, y2 = x1 + width, y1 + height
    cropped = cv2.resize(data[y1:y2, x1:x2], (128, 128))
    res.append(cropped)
  return res

## ii) Test File Load

In [ ]:
path = '/content/drive/MyDrive/Focus on Me/movie'
file_name = os.listdir(path)

## iii) File Sort

In [ ]:
def last_8chars(x):
    return(x[-8:])

file_name = sorted(file_name, key = last_8chars)   

## iv) Face Crop

In [ ]:
final = []
for i in range(0,len(file_name)) :
  res=get_cropimg(path + '/' + file_name[i])
  final.append(res)
len(final)

In [ ]:
lists = []

for i in final:
  for j in range(len(i)) :
    lists.append(i[j])

print(len(lists))

In [ ]:
!pip install deepface
from deepface import DeepFace
from deepface.commons import functions
backends = ['opencv', 'ssd', 'dlib', 'mtcnn']

In [ ]:
i = 
j = 
test_en_1 = face_recognition.face_encodings(lists[i],face_box,num_jitters=1000,model='large')
test_en_2 = face_recognition.face_encodings(lists[j],face_box,num_jitters=1000,model='large')
test_en_1 = np.array(test_en_1)
test_en_2 = np.array(test_en_2)
face_recognition.face_distance(test_en_2, test_en_1)
plt.subplot(211)
plt.imshow(lists[i])
plt.subplot(lists[j])

## v) Face Encoding

In [ ]:
encode = []
face_box = [(0,64,64,0)]
for i in range(0,len(lists)) :
  temp = face_recognition.face_encodings(lists[i],face_box,num_jitters=1000,model='large')
  encode.append(temp)

In [ ]:
len(encode)

In [ ]:
face_recognition.face_distance(np.array(encode[551]), np.array(encode[547]))

## vi) Face 특징값 거리 계산

In [ ]:
for i in range(0,3) :
  for j in range(0,250) :
    distance = face_recognition.face_distance(np.array(encode[i]), np.array(encode[j]))
    if distance <= 0.4 :
      print(i,j)
    else :
      pass

In [ ]:
len(encode)

## vii) 중복인물 검출

In [ ]:
person_test = {}
for i in range(0,100) :
  same = []
  if i not in itertools.chain(*list(person_test.values())) :
    for j in range(0,100) :
      distance = face_recognition.face_distance(np.array(encode[i]), np.array(encode[j]))
      if distance <= 0.3 :
        same.append(j)
        person_test[i] = same
    else :
      pass
  else :
    pass

In [ ]:
for key, value in  person.items() :
    if 93 in value :
      print(key)

In [ ]:
face_recognition.face_distance(np.array(encode[93]), np.array(encode[3])), face_recognition.face_distance(np.array(encode[93]), np.array(encode[31])),face_recognition.face_distance(np.array(encode[93]), np.array(encode[37])),face_recognition.face_distance(np.array(encode[93]), np.array(encode[56]))

In [ ]:
i = 93
j = 56
test_en_1 = face_recognition.face_encodings(lists[i],face_box,num_jitters=1000,model='large')
test_en_2 = face_recognition.face_encodings(lists[j],face_box,num_jitters=1000,model='large')
test_en_1 = np.array(test_en_1)
test_en_2 = np.array(test_en_2)
face_recognition.face_distance(test_en_2, test_en_1)
plt.subplot(221)
plt.imshow(lists[i])
plt.subplot(222)
plt.imshow(lists[j])

In [ ]:
len(np.concatenate(list(person.values())))

In [ ]:
sorted(np.concatenate(list(person.values())))

In [ ]:
len(person.keys())

# V. SQL 연동

## i) DB 연동 및 Face Binary Load

In [ ]:
db = pymysql.connect(host='yangjae-team08-database.ca8iiefanafw.eu-south-1.rds.amazonaws.com', port = 3306, user = 'admin', password = 'yangjae8', db='mydb', charset = 'utf8')

In [ ]:
sql = """
    select f.face_array, t.ad_id, t.gender, t.age 
    from face f, target t
    WHERE f.face_id = t.face_id
"""

In [ ]:
cursor = db.cursor()
cursor.execute(sql)

In [ ]:
sql = """
    select f.face_array, t.ad_id, t.gender, t.age , f.created_at
    from face f, target t
    WHERE f.face_id = t.face_id
"""

In [ ]:
a = pd.read_sql(sql,db)

In [ ]:
a

## ii) Binary Decode

In [ ]:
from PIL import Image
import base64
import io
import numpy as np
import torch

data = a.loc[704].values[0]
base64_decoded = base64.b64decode(data)

image = Image.open(io.BytesIO(base64_decoded))
image_np = np.array(image)

plt.imshow(image)

In [ ]:
face_array = []
bin = a.loc[3].values[0]
base64_decoded = base64.b64decode(bin)
image = Image.open(io.BytesIO(base64_decoded))
image_np = np.array(image)
face_array.append(image_np)
plt.imshow(face_array[0])

In [ ]:
face_array = []
for i in range(0,len(a)) :
  bin = a.loc[i].values[0]
  base64_decoded = base64.b64decode(bin)
  image = Image.open(io.BytesIO(base64_decoded))
  image_np = np.array(image)
  face_array.append(image_np)

## iii) 특징값 추출 리스트

In [ ]:
encode_db = []
face_box = [(0,128,128,0)]
for i in range(0,len(face_array)) :
  temp = face_recognition.face_encodings(face_array[i],face_box,num_jitters=1000,model='large') # num_jitters 값 변경해서 성능 향상 할 수 있음
  encode_db.append(temp)

## iv) 동일인물 모인 딕셔너리

In [ ]:
person = {}
for i in range(0,len(encode_db)) :
  same = []
  if i not in itertools.chain(*list(person.values())) :
    for j in range(0,len(encode_db)) :
      distance = face_recognition.face_distance(np.array(encode_db[i]), np.array(encode_db[j]))
      if distance <= 0.35 :
        same.append(j)
        person[i] = same
    else :
      pass
  else :
    pass

In [ ]:
person

In [ ]:
i = 660
j = 655
plt.subplot(221)
plt.imshow(face_array[i])
plt.subplot(222)
plt.imshow(face_array[j])
# plt.subplot(223)
# plt.imshow(face_array[k])

In [ ]:
person_value = person.values()
person_key = person.keys()
person_value = list(person_value)
person_key = list(person_key)

print(person_value)
print(person_key)
print(len(person_value))
print(len(person_key))

In [ ]:
a = pd.read_sql(sql,db)

In [ ]:
array_df = a

In [ ]:
a

In [ ]:
# index번호를 값으로 갖는 column 생성
a.reset_index(inplace=True)
# 중복인물 제거 (person_key리스트에 있는 값만 추출)
unique = a[a['index'].isin(person_key)]

In [ ]:
unique

## v) 중복인물 검출 결과 이용한 광고 효율성 지표 계산

In [ ]:
# 2030 여성의 수
w2030 = len(unique[(unique['gender']=='woman') & (unique['age']=='2030')])
print('2030 여성: ', w2030, '명')

# 4번 광고를 한번 이상 본 2030 여성
w2030_ad4 = len(unique[(unique['gender']=='woman') & (unique['age']=='2030') & (unique['ad_id']==4)]) 
print('Market Curly 광고를 시청한 2030 여성: ', w2030_ad4, '명')

# 4번 광고의 reach 값 :
reach_ad4 = round(w2030_ad4/w2030*100, 2)
print('Market Curly 광고의 reach(도달률): ', reach_ad4, '%')

In [ ]:
# 4번 광고의 frequency : 
ad4_cnt = len(a[(a['gender']=='woman') & (a['age']=='2030') & (a['ad_id']==4)])
freq_ad4 = round(ad4_cnt/w2030_ad4, 2)
print('Market Curly 광고의 frequency(빈도): ', freq_ad4, '회')

In [ ]:
# 2030 여성의 다른 광고 reach
w2030_ad3 = len(unique[(unique['gender']=='woman') & (unique['age']=='2030') & (unique['ad_id']==3)]) 
reach_ad3 = round(w2030_ad3/w2030*100, 2)
print(reach_ad3, '%')

In [ ]:
# 2030 남성의 수
m2030 = len(unique[(unique['gender']=='man') & (unique['age']=='2030')])
print('2030 남성: ', m2030, '명')

# 3번 광고를 한번 이상 본 2030 남성
m2030_ad3 = len(unique[(unique['gender']=='man') & (unique['age']=='2030') & (unique['ad_id']==3)]) 
print('Musinsa 광고를 시청한 2030 남성: ', m2030_ad3, '명')

# 3번 광고의 reach 값 :
reach_ad3 = round(m2030_ad3/m2030*100, 2)
print('Musinsa 광고의 reach(도달률): ', reach_ad3, '%')

In [ ]:
# 3번 광고의 frequency : 
ad3_cnt = len(a[(a['gender']=='man') & (a['age']=='2030') & (a['ad_id']==3)])
freq_ad3 = round(ad3_cnt/m2030_ad3, 2)
print('Musinsa 광고의 frequency(빈도): ', freq_ad3, '회')